FineTuning

In [ ]:
import torch
from momentfm import MOMENTPipeline
from momentfm.data.classification_dataset import ClassificationDataset
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import CosineAnnealingLR
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tqdm import tqdm
import warnings
import pandas as pd
import numpy as np
import time
from aeon.datasets import write_to_ts_file
from sklearn.model_selection import train_test_split
set_seed = 42
torch.manual_seed(set_seed)
np.random.seed(set_seed)

warnings.filterwarnings("ignore", category=UserWarning, message=".*use_reentrant.*")

device = "cuda" if torch.cuda.is_available() else "cpu"

# Load data
df = pd.read_csv("./Dataset/Classification/60_min/comstock_60min_small.csv").drop("Timestamp", axis=1)
X = df.T  
df_label_all = pd.read_csv("./Dataset/Classification/60_min/comstock_60min_labels.csv")
appliance_names = ['cooling_ON', 'fans_ON', 'heat_rejection_ON', 'heating_ON', 'refrigeration_ON', 'water_systems_ON']
df_label = df_label_all[appliance_names]
y = df_label.values  

results = []  # Store results per appliance

for appliance in appliance_names:
    print(f"\n⚡ Starting fine-tuning for appliance: {appliance}")
    start_time = time.time()
    # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

    y = df_label_all[appliance]
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, stratify=y, random_state=42
    )

    X_train = np.array(X_train)[:, np.newaxis, :]
    X_test = np.array(X_test)[:, np.newaxis, :]
    y_train = np.array(y_train)
    y_test = np.array(y_test)

    write_to_ts_file(X_train, '/home/user/Downloads/moment/dataset/', y_train,
                    problem_name='Comstock_TRAIN.ts', header=None, regression=False)

    write_to_ts_file(X_test, '/home/user/Downloads/moment/dataset/', y_test,
                    problem_name='Comstock_TEST.ts', header=None, regression=False)

    train_dataset = ClassificationDataset(data_split='train')
    test_dataset = ClassificationDataset(data_split='test')

    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, drop_last=False)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, drop_last=False)

    model = MOMENTPipeline.from_pretrained(
        "AutonLab/MOMENT-1-large", 
        model_kwargs={
            'task_name': 'classification',
            'n_channels': 1,
            'num_class': 2,
            'freeze_encoder': False,
            'freeze_embedder': False,
            'reduction': 'mean',
        },
    )
    model.init()
    model.to(device).float()

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-4)
    T_MAX = max(int((700 / 168) / 32 * 10), 1)
    scheduler = CosineAnnealingLR(optimizer, T_max=T_MAX, eta_min=1e-7)

    best_test_acc = 0.0
    final_metrics = {}

    for epoch in range(10):
        model.train()
        running_loss = 0.0
        correct_preds = 0
        total_preds = 0

        for batch_x, batch_masks, batch_labels in tqdm(train_loader):
            batch_x = batch_x.to(device).float().requires_grad_()
            batch_masks = batch_masks.to(device).long()
            batch_labels = batch_labels.to(device).long()
            batch_masks = torch.ones(batch_x.size(0), batch_x.size(-1)).long().to(device)

            output = model(x_enc=batch_x, input_mask=batch_masks, reduction='mean')
            logits = output.logits

            loss = criterion(logits, batch_labels)

            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_value_(model.parameters(), clip_value=5.0)
            optimizer.step()

            _, predicted = torch.max(logits, 1)
            correct_preds += (predicted == batch_labels).sum().item()
            total_preds += batch_labels.size(0)

            running_loss += loss.item()

        scheduler.step()

        # Evaluation
        model.eval()
        all_preds = []
        all_labels = []
        correct_test_preds = 0
        total_test_preds = 0

        with torch.no_grad():
            for batch_x, batch_masks, batch_labels in test_loader:
                batch_x = batch_x.to(device).float()
                batch_masks = batch_masks.to(device).long()
                batch_labels = batch_labels.to(device).long()

                output = model(x_enc=batch_x, input_mask=batch_masks, reduction='mean')
                logits = output.logits

                _, predicted = torch.max(logits, 1)
                correct_test_preds += (predicted == batch_labels).sum().item()
                total_test_preds += batch_labels.size(0)

                all_preds.append(predicted.cpu().numpy())
                all_labels.append(batch_labels.cpu().numpy())

        all_preds = np.concatenate(all_preds)
        all_labels = np.concatenate(all_labels)

        test_acc = correct_test_preds / total_test_preds
        test_prec = precision_score(all_labels, all_preds, zero_division=0)
        test_rec = recall_score(all_labels, all_preds, zero_division=0)
        test_f1 = f1_score(all_labels, all_preds, zero_division=0, average='macro')

        print(f"Epoch {epoch+1}: Test Acc={test_acc:.4f}, Precision={test_prec:.4f}, Recall={test_rec:.4f}, F1={test_f1:.4f}")

        if test_acc > best_test_acc:
            best_test_acc = test_acc
            final_metrics = {
                'Appliance': appliance,
                'Accuracy': test_acc,
                'Precision': test_prec,
                'Recall': test_rec,
                'F1_Score': test_f1
            }
            # torch.save(model.state_dict(), f"/home/user/Downloads/moment/models/best_moment_{appliance}.pth")

    elapsed = time.time() - start_time
    print(f"✅ Finished {appliance} in {elapsed:.2f} seconds.")

    results.append(final_metrics)

# Save results as CSV
results_df1 = pd.DataFrame(results)
results_df1.insert(1, 'Freq', 60)  # Add frequency column
results_df1
# results_df.to_csv("/home/user/Downloads/moment/Classification_results/comstock_60min_fine_tune_results_try.csv", index=False)
# print("\n All appliance-wise results saved as CSV!")

In [ ]:
import os
import torch
import time
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import CosineAnnealingLR
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from momentfm import MOMENTPipeline
from momentfm.data.classification_dataset import ClassificationDataset
from aeon.datasets import write_to_ts_file
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


device = "cuda" if torch.cuda.is_available() else "cpu"
df = pd.read_csv("./Dataset/Classification/60_min/comstock_60min_small.csv").drop("Timestamp", axis=1)
X = df.T  
df_label_all = pd.read_csv("./Dataset/Classification/60_min/comstock_60min_labels.csv")
appliance_names = ['cooling_ON', 'fans_ON', 'heat_rejection_ON', 'heating_ON', 'refrigeration_ON', 'water_systems_ON']
df_label = df_label_all[appliance_names]
y = df_label.values  
Num_time_series = 700
length_time_series = 168
batch_size = 32
epochs = 10

results = []  # Store results per appliance

for appliance in appliance_names:
    print(f"\n⚡ Starting fine-tuning for appliance: {appliance}")
    start_time = time.time()

    # Get the data for the current appliance
    y = df_label_all[appliance]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

    X_train = np.array(X_train)[:, np.newaxis, :]
    X_test = np.array(X_test)[:, np.newaxis, :]
    y_train = np.array(y_train)
    y_test = np.array(y_test)

    # Save data as .ts files
    write_to_ts_file(X_train, '/home/user/Downloads/moment/dataset/', y_train, problem_name='Comstock_TRAIN.ts', header=None, regression=False)
    write_to_ts_file(X_test, '/home/user/Downloads/moment/dataset/', y_test, problem_name='Comstock_TEST.ts', header=None, regression=False)

    train_dataset = ClassificationDataset(data_split='train')
    test_dataset = ClassificationDataset(data_split='test')

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, drop_last=False)

    model = MOMENTPipeline.from_pretrained(
        "AutonLab/MOMENT-1-large", 
        model_kwargs={
            'task_name': 'classification',
            'n_channels': 1,
            'num_class': 2,
            'freeze_encoder': False,
            'freeze_embedder': False,
            'reduction': 'mean',
        },
    )
    model.init()
    model.to(device).float()

    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
    T_MAX = max(int((Num_time_series / length_time_series) / batch_size * epochs), 1)
    scheduler = CosineAnnealingLR(optimizer, T_max=T_MAX, eta_min=1e-7)

    best_test_acc = 0.0
    final_metrics = {}

    for epoch in range(10):
        model.train()
        running_loss = 0.0
        correct_preds = 0
        total_preds = 0

        for batch_x, batch_masks, batch_labels in tqdm(train_loader):
            batch_x = batch_x.to(device).float().requires_grad_()
            batch_masks = batch_masks.to(device).long()
            batch_labels = batch_labels.to(device).long()
            batch_masks = torch.ones(batch_x.size(0), batch_x.size(-1)).long().to(device)

            output = model(x_enc=batch_x, input_mask=batch_masks, reduction='mean')
            logits = output.logits

            loss = criterion(logits, batch_labels)

            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_value_(model.parameters(), clip_value=5.0)
            optimizer.step()

            _, predicted = torch.max(logits, 1)
            correct_preds += (predicted == batch_labels).sum().item()
            total_preds += batch_labels.size(0)

            running_loss += loss.item()

        scheduler.step()

        # Evaluation
        model.eval()
        all_preds = []
        all_labels = []
        correct_test_preds = 0
        total_test_preds = 0

        with torch.no_grad():
            for batch_x, batch_masks, batch_labels in test_loader:
                batch_x = batch_x.to(device).float()
                batch_masks = batch_masks.to(device).long()
                batch_labels = batch_labels.to(device).long()

                output = model(x_enc=batch_x, input_mask=batch_masks, reduction='mean')
                logits = output.logits

                _, predicted = torch.max(logits, 1)
                correct_test_preds += (predicted == batch_labels).sum().item()
                total_test_preds += batch_labels.size(0)

                all_preds.append(predicted.cpu().numpy())
                all_labels.append(batch_labels.cpu().numpy())

        all_preds = np.concatenate(all_preds)
        all_labels = np.concatenate(all_labels)

        test_acc = correct_test_preds / total_test_preds
        test_prec = precision_score(all_labels, all_preds, zero_division=1)
        test_rec = recall_score(all_labels, all_preds, zero_division=1)
        test_f1 = f1_score(all_labels, all_preds, zero_division=1, average='macro')

        print(f"Epoch {epoch+1}: Test Acc={test_acc:.4f}, Precision={test_prec:.4f}, Recall={test_rec:.4f}, F1={test_f1:.4f}")

        if test_acc > best_test_acc:
            best_test_acc = test_acc
            final_metrics = {
                'Appliance': appliance,
                'Accuracy': test_acc,
                'Precision': test_prec,
                'Recall': test_rec,
                'F1_Score': test_f1
            }

    elapsed = time.time() - start_time
    print(f"✅ Finished {appliance} in {elapsed:.2f} seconds.")

    results.append(final_metrics)

results_df1 = pd.DataFrame(results)
print(f"\nAll appliance-wise results saved")